# GödelOS Demonstration Notebook

This notebook demonstrates the advanced capabilities of the GödelOS system, focusing on its intelligent test runner, automated statistics collection, and insightful graphical outputs.

GödelOS is a modular, AI-powered operating system for knowledge, reasoning, and metacognition research. Here, we showcase its:
- Automated test discovery and execution
- Rich statistics collection (pass/fail rates, timing, trends)
- Visual analytics for rapid insight

Let's get started!

## Setup

Ensure the following Python packages are installed in your environment:
- `matplotlib`
- `seaborn`
- `pandas`

If running for the first time, you can install them using:
```python
!pip install matplotlib seaborn pandas
```

Also ensure GödelOS is installed and available in your Python path.

In [ ]:
# Import core libraries
!source godel_venv/bin/activate
!pip3.13 install -r "requirements.txt"
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from godelOS.test_runner.test_runner import TestRunner
from godelOS.test_runner.statistics_collector import TestStatistics

# Set up plotting aesthetics
sns.set_theme(style="whitegrid")

## Run GödelOS Test Suite

We instantiate the test runner, discover, and execute all available tests in the project.

In [ ]:
# Initialize TestRunner (uses default config)
runner = TestRunner()

# Discover and categorize tests
runner.discover_tests()
runner.categorize_tests()

# Run all tests
runner.run_tests()

## Collect and Display Test Statistics

In [ ]:
# Get statistics from the run
stats: TestStatistics = runner.statistics_collector.calculate_statistics()

# Display basic statistics
print(f"Total tests: {stats.total}")
print(f"Passed: {stats.passed}")
print(f"Failed: {stats.failed}")
print(f"Skipped: {stats.skipped}")
print(f"Errors: {stats.error}")
print(f"Pass rate: {stats.pass_rate:.2%}")

## Visualize Test Outcomes

Let's visualize the distribution of test outcomes for a quick overview.

In [ ]:
# Pie chart of test outcomes
labels = ['Passed', 'Failed', 'Skipped', 'Error']
sizes = [stats.passed, stats.failed, stats.skipped, stats.error]
colors = sns.color_palette('pastel')[0:4]

plt.figure(figsize=(6,6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.title('GödelOS Test Outcomes')
plt.show()

## Timing Analysis

Examine durations of tests to identify performance bottlenecks and outliers.

In [ ]:
# Histogram of test durations
durations = [d for _, d in getattr(stats, 'slowest_tests', [])]
if not durations:
    # Fallback: use all durations if slowest_tests is empty
    durations = []
    for cat, tests in runner.results_collector.get_all_results().items():
        for r in tests.values():
            if hasattr(r, 'duration') and r.duration is not None:
                durations.append(r.duration)

if durations:
    plt.figure(figsize=(8,4))
    sns.histplot(durations, bins=20, kde=True, color='skyblue')
    plt.xlabel('Test Duration (s)')
    plt.ylabel('Number of Tests')
    plt.title('Distribution of Test Durations')
    plt.show()
else:
    print('No timing data available.')

## Slowest Tests

Highlight the slowest tests for targeted optimization.

In [ ]:
# Bar chart of slowest tests
slow_tests = getattr(stats, 'slowest_tests', [])
if slow_tests:
    df = pd.DataFrame(slow_tests, columns=['Test', 'Duration (s)'])
    plt.figure(figsize=(10, 5))
    sns.barplot(y='Test', x='Duration (s)', data=df.sort_values('Duration (s)', ascending=False), palette='flare')
    plt.title('Top Slowest Tests')
    plt.xlabel('Duration (s)')
    plt.ylabel('Test Name')
    plt.tight_layout()
    plt.show()
else:
    print('No slowest test data available.')

## Most Failing Tests

Identify tests that fail most frequently to guide debugging efforts.

In [ ]:
fail_tests = getattr(stats, 'most_failing_tests', [])
if fail_tests:
    df = pd.DataFrame(fail_tests, columns=['Test', 'Failure Count'])
    plt.figure(figsize=(10, 5))
    sns.barplot(y='Test', x='Failure Count', data=df.sort_values('Failure Count', ascending=False), palette='rocket')
    plt.title('Most Failing Tests')
    plt.xlabel('Failure Count')
    plt.ylabel('Test Name')
    plt.tight_layout()
    plt.show()
else:
    print('No failing test data available.')

## Category-wise Test Statistics

Analyze pass/fail rates by test category for deeper insight.

In [ ]:
cat_stats = getattr(stats, 'category_stats', {})
if cat_stats:
    df = pd.DataFrame([
        {'Category': cat, 'Total': s.total, 'Passed': s.passed, 'Failed': s.failed, 'Skipped': s.skipped}
        for cat, s in cat_stats.items()
    ])
    df = df.set_index('Category')
    df[['Passed', 'Failed', 'Skipped']].plot(kind='bar', stacked=True, figsize=(10,6), colormap='tab20')
    plt.title('Test Outcomes by Category')
    plt.ylabel('Number of Tests')
    plt.xlabel('Category')
    plt.tight_layout()
    plt.show()
else:
    print('No category statistics available.')

## Conclusion

This notebook has demonstrated the automation and insight provided by GödelOS's test runner and analytics modules. With programmatic access to test statistics and rich visualizations, GödelOS enables rapid, data-driven development and debugging.